In [3]:
import pandas as pd
import tweepy #https://github.com/tweepy/tweepy
import csv
from datetime import datetime
import os
import re
import requests
import twitter
import lxml.html
import time

In [22]:
ironhack= pd.read_csv('ironhack_twitter_community.csv')
ironhack.describe()
ironhack_q3 = ironhack[(ironhack['protected']==False)&(ironhack['statuses_count']>=793)&(ironhack['statuses_count']<=3307)]
ironhack_q3.shape

(2890, 14)

In [4]:
testers = pd.read_csv('ironhack_personality_testers.csv')

In [5]:
#Twitter API credentials
consumerkey = 'EV9ulEXQ23ZxK8CzQqpSH5S26'
consumersecret = 'AIEMTk197UwFQRl1IEqGmBgz8fONJoJaI6RjW59H9EvHX261cY'
accesskey = '1119208863574708224-R0rPmcbD8ReYeak6u4Qeu7Idv0cPeI'
accesssecret = 'pgl972ioi4RrWcxkyUmB7TvuNsr0nJOLRpL8oX2rgqyrE'

auth = tweepy.OAuthHandler(consumerkey, consumersecret)
auth.set_access_token(accesskey, accesssecret)
api = tweepy.API(auth)


In [6]:
t = twitter.Api(
    consumer_key = consumerkey,
    consumer_secret = consumersecret,
    access_token_key = accesskey, 
    access_token_secret = accesssecret,
    tweet_mode='extended' # this ensures that we get the full text of the users' original tweets
)

In [23]:
def first_200(name): #working
    sample = t.GetUserTimeline(screen_name=name, count=200)
    return sample

def testing_first_200(name): #working
    sample = t.GetUserTimeline(screen_name=name, count=200)
    text = ''
    hashtags = []
    for tweet in sample:
        text += tweet.full_text
        hashtags.append(clean_hashtags(tweet.hashtags))
    output = [text, hashtags]   
    return output

def get_tweets(name):
    all_tweets = []
    sample = first_200(name)
    all_tweets.extend(sample)
    last_id = sample[-1].id
    for i in range(1):
        time.sleep(1)
        new = t.GetUserTimeline(screen_name=name, max_id=last_id-1)
        if len(new) > 0:
            all_tweets.extend(new)
            last_id = new[-1].id
        else:
            break
    
    return all_tweets

def clean_hashtags(hashtags):
    """
    Turns data with any number of hashtags like this - [Hashtag(Text='STEMonStation')] - to a list like this -
    ['STEMonStation']
    """
    cleaned = []
    if len(hashtags) >= 1:
        for i in range(len(hashtags)):
            cleaned.append(hashtags[i].text)        
    return cleaned


def full_tweet_text_hashtag(name) : 
    tweets_expanded = get_tweets(name)
    
    text = ''
    hashtags = []
    
    for tweet in tweets_expanded :
            text += tweet.full_text
            hashtags.append(clean_hashtags(tweet.hashtags))
    output = [text, hashtags]
            
    return output


In [8]:
testers['apply_function_test'] = testers['screen_name'].apply(testing_first_200)

In [10]:
testers['apply_function_test'][0]

['@yuris That’s the right spirit. Never miss a good crisis!Facebook acquires the #VR game studio behind one of the Rift\'s best titles https://t.co/Fw7SygaRJWRT @FIBerlin: Let\'s bring YOU and YOUR IDEA into BUSINESS. Get all of your questions answered to see if FI is the right fit for you. Applic…RT @RtoVR: 2019 Was a Major Inflection Point for VR—Here\'s the Proof https://t.co/N74SnQxivN @tipatat https://t.co/5fezyHkZw5@InflightVR @evelopair @Airbus Love it. Congrats!RT @qatarairways: #QatarAirways Business passengers can experience a virtual ‘World Like Never Before’ as the #OryxOne in-flight entertainm…RT @UFOstart: 🚀How do you grow a community and gain mass adoption? Check out our new post on how to communicate your brand: https://t.co/eb…@ringsdorff @salesforce @newstoreinc Super strong. Congrats!RT @InflightVR: BRICK-BUILT VR! Stoked to collaborate with @SunExpress @LEGO @PicoInteractive @CrossLasker on a branded, immersive #VR #Pax…RT @InflightVR: 4K #VR headsets with the lates

In [ ]:
ironhack_q3['apply_function_test'] = ironhack_q3['screen_name'].apply(testing_first_200)

In [18]:
get_tweets('realDonaldTrump')

[Status(ID=1239650086491230209, ScreenName=realDonaldTrump, Created=Mon Mar 16 20:29:46 +0000 2020, Text='I ask all Americans to band together and support your neighbors by not hoarding unnecessary amounts of food and essentials. TOGETHER we will stay STRONG and overcome this challenge!')]

In [ ]:
testers
